In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from models.ResUNet import *


In [2]:
images = np.load("/media/bot/DANE/MED_DATASETS/NARZ_ZAGRO2/FILES/imgs16_half.npy")
labels = np.load("/media/bot/DANE/MED_DATASETS/NARZ_ZAGRO2/FILES/masks16_half.npy")


N = images.shape[0]
W = images.shape[1]
H = images.shape[2]
C = labels.shape[3]

In [3]:
x_train, x_test_valid, y_train, y_test_valid = train_test_split(images, labels, test_size=0.4, random_state=4)
x_valid, x_test, y_valid, y_test = train_test_split(x_test_valid, y_test_valid, test_size=0.5, random_state=4)

print(x_train.shape, y_train.shape)

(1020, 256, 256, 1) (1020, 256, 256, 5)


In [4]:
seg_model = ResUNet(img_shape = (H, W, 1), num_of_class = C,learning_rate = 2e-4, path = "saved_models/UNet_model_300_100", do_drop = True, drop_rate = 0.5)
seg_model.build_model()
seg_model.show_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
add (Add)                       (None, 256, 256, 64) 0           conv2d_1[0][0]                   
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         add[0][0]                    

In [ ]:
history = seg_model.train(x_train, y_train, x_valid, y_valid, epoch = 100, batch_size = 8)

Epoch 1/100
128/128 [==============================] - 2051s 16s/step - loss: 57.2056 - dice_coef_multilabel: 0.0391 - iou_multilabel: 0.0214 - val_loss: 2.6482 - val_dice_coef_multilabel: 0.1096 - val_iou_multilabel: 0.0641
INFO:tensorflow:Assets written to: saved_models/ResUNet_smaller_imgs/assets
Epoch 2/100
128/128 [==============================] - 2104s 16s/step - loss: 2.7202 - dice_coef_multilabel: 0.1163 - iou_multilabel: 0.0681 - val_loss: 1.6108 - val_dice_coef_multilabel: 0.1393 - val_iou_multilabel: 0.0843
INFO:tensorflow:Assets written to: saved_models/ResUNet_smaller_imgs/assets
Epoch 3/100
128/128 [==============================] - 2542s 20s/step - loss: 1.9510 - dice_coef_multilabel: 0.1335 - iou_multilabel: 0.0803 - val_loss: 4.4264 - val_dice_coef_multilabel: 0.1134 - val_iou_multilabel: 0.0664
Epoch 4/100
128/128 [==============================] - 2351s 18s/step - loss: 3.0232 - dice_coef_multilabel: 0.1254 - iou_multilabel: 0.0765 - val_loss: 2.6056 - val_dice_coef

In [4]:
loaded_model = tf.keras.models.load_model('saved_models/ResUNet_smaller_imgs', custom_objects={"dice_coef_multilabel": dice_coef_multilabel, "iou_multilabel": iou_multilabel})

preds_train = loaded_model.predict(x_train)
print("TRAIN DICE_COEF: ", dice_coef_multilabel(y_train.astype('float32'), preds_train).numpy())
print("TRAIN IoU: ", iou_multilabel(y_train.astype('float32'), preds_train, 5).numpy())

preds_test = loaded_model.predict(x_test)
print("TEST DICE_COEF: ", dice_coef_multilabel(y_test.astype('float32'), preds_test).numpy())
print("TEST IoU: ", iou_multilabel(y_test.astype('float32'), preds_test, 1).numpy())

TRAIN DICE_COEF:  0.24619496
TRAIN IoU:  0.17802517
TEST DICE_COEF:  0.2441899
TEST IoU:  0.0


In [5]:
print("TEST DICE_COEF, IOU Spinal cord: ", dice_coef(y_test.astype('float32')[:, :, :, 0], preds_test[:, :, :, 0]).numpy(), iou(y_test.astype('float32')[:, :, :, 0], preds_test[:, :, :, 0]).numpy())
print("TEST DICE_COEF, IOU Left lung: ", dice_coef(y_test.astype('float32')[:, :, :, 1], preds_test[:, :, :, 1]).numpy(), iou(y_test.astype('float32')[:, :, :, 1], preds_test[:, :, :, 1]).numpy())
print("TEST DICE_COEF, IOU Right lung: ", dice_coef(y_test.astype('float32')[:, :, :, 2], preds_test[:, :, :, 2]).numpy(), iou(y_test.astype('float32')[:, :, :, 2], preds_test[:, :, :, 2]).numpy())
print("TEST DICE_COEF, IOU Heart: ", dice_coef(y_test.astype('float32')[:, :, :, 3], preds_test[:, :, :, 3]).numpy(), iou(y_test.astype('float32')[:, :, :, 3], preds_test[:, :, :, 3]).numpy())
print("TEST DICE_COEF, IOU Esophagus: ", dice_coef(y_test.astype('float32')[:, :, :, 4], preds_test[:, :, :, 4]).numpy(), iou(y_test.astype('float32')[:, :, :, 4], preds_test[:, :, :, 4]).numpy())

TEST DICE_COEF, IOU Spinal cord:  6.079435e-12 0.0
TEST DICE_COEF, IOU Left lung:  0.5804844 0.4089313
TEST DICE_COEF, IOU Right lung:  0.64005643 0.4706493
TEST DICE_COEF, IOU Heart:  5.429907e-13 0.0
TEST DICE_COEF, IOU Esophagus:  0.00040880367 0.00020444363
